In [ ]:
import ipywidgets.widgets as widgets
import logging
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.controller import link_controller
from ballsbot.manipulator import Manipulator
from ballsbot.joystick import JoystickWrapper
from ballsbot.drawing import ManipulatorPoseDrawing
from ballsbot.dashboard import Dashboard
from ballsbot.utils import run_as_thread, keep_rps
from ballsbot.ros_messages import get_ros_messages
from ballsbot.magnetic_encoders import MagneticEncoders

In [ ]:
get_ros_messages().start(sync=False)

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

dashboard = Dashboard(widgets)

manipulator_drawing_raw = ManipulatorPoseDrawing(dashboard, 'manipulator_xy', 'manipulator_xz')
dashboard.add_subplot('manipulator_xy', manipulator_drawing_raw.get_drawing_func_name())
dashboard.add_subplot('manipulator_xz', manipulator_drawing_raw.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
logging.basicConfig(
    format='[%(asctime)s] %(levelname)s %(name)s: %(message).700s',
    filename='/tmp/manipulator.log',
    filemode='w',
    level=logging.INFO,
)

In [ ]:
joystick_wrapper = JoystickWrapper(controller)
link_controller(joystick_wrapper)
manipulator = Manipulator(joystick_wrapper)

encoders = MagneticEncoders()
encoders.start()

In [ ]:
def update_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=2)

        pose = manipulator.get_manipulator_pose()
        manipulator_drawing_raw.update_image(pose, override_crop_half_size=750.)
        
        dashboard.redraw()

run_as_thread(update_images)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()